# OpenAI Distillation and Fine-tuning
A basic fine-tuning/distillation tutorial notebook by [Trelis Research](https://trelis.com/about).


- Watch the [basic video on YouTube]().
- Watch the [advanced video on YouTube](https://youtu.be/iogrvDu5K0k).
- Get the advanced notebook at [Trelis.com/ADVANCED-fine-tuning](https://Trelis.com/ADVANCED-fine-tuning).
- Get more Trelis Resources at [Trelis.com/About]().

## Introduction / Explanation

### Why fine-tune / distill?

A. OpenAI models lack the knowledge for your specific application AND using retrieval methods (i.e. adding background information to prompts) is giving bad results.

B. OpenAI models are responding with the wrong format AND prompt engineering is giving bad results.

C. Strong models (GPT-4o) gives good results, but is too SLOW or EXPENSIVE.

### What is Distillation vs Fine-tuning?

#### Training Stages:
-- Pre-training: Using large amounts of raw / minimally processed data.

-- Fine-tuning: Training a pre-trained model with small amounts of curated data.

*You can use distillation for either stage!*


#### Training Approaches:
-- Data Based Approach, i.e. you have to prepare lots of data.

-- Distillation Approach: Take a short cut by using a stronger model (either to create the data AND/OR to by copying the weights directly or indirectly.


#### Distillation Approaches:
-- Black box: You use a stronger model to generate data. WHAT WE DO IN THIS NOTEBOOK.

-- White box: You have access to the stronger model's inner workings. You compare the probability distributions of the stronger model with the weaker model, and adjust the weaker model. Deeper YouTube Video [Here](https://youtu.be/tf60owmwR-c)

### Simple Approach to Distilling / Fine-tuning

0. Evaluate the performance of a strong and weak model.
1. Generate synthetic data using a strong model (ideally using augmented data).
2. Use that data to fine-tuning a cheaper/faster model (e.g. mini).
3. Run an evaluation to see if the fine-tuning helped.

### ADVANCED Approach to Distilling / Fine-tuning

Get the advanced notebook at [Trelis.com/ADVANCED-fine-tuning](https://Trelis.com/ADVANCED-fine-tuning) and watch the video on [YouTube](https://youtu.be/iogrvDu5K0k).

Includes:
- **Augmented Synthetic Question and Answer Generation**:
  - Diverse questions, augmented with background documentation.
  - Accurate answers:
    - Sampling at low temperature.
    - Filtering of correct/strong responses.
  
- **Automatic Evaluations**: Judging of results.

## Installation

In [ ]:
!python -m pip install --upgrade pip -q
!pip install openai -qU

In [ ]:
!pip show openai

Name: openai
Version: 1.51.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
import os
import getpass
from openai import OpenAI

from google.colab import userdata


# Set OpenAI API key
api_key = userdata.get('OPENAI_API_KEY')

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

## Set up Questions and Evaluate Performance

In [ ]:
# Define the training questions
train_questions = [
    "Is a try worth one or five points in touch rugby?",
    "How much is a try worth in touch rugby?",
    "Is a try worth five points in touch rugby?",
    "What happens after a try is scored in touch rugby?",
    "How is the game restarted after a try in touch rugby?",
    "What is the role of the referee in touch rugby?",
    "Are there any differences between touch rugby and rugby union?",
    "What is the maximum number of players on each team in touch rugby?",
    "How long is a typical touch rugby match?",
    "How many players on the field in touch rugby?"
]

In [ ]:
import openai

# Function to call OpenAI and print the response
def get_openai_response(model_name, question):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a sports rules expert."},
            {"role": "user", "content": question}
        ],
        temperature=0.25,
        top_p=0.9 # help avoid bad tokens when sampling.
    )
    print(response.choices[0].message.content)
    print()

def evaluate(model_name,qs_to_eval=10):
  print(f"Generating with model: {model_name}:")
  # Call the function with both models
  for question in train_questions[:qs_to_eval]:
      print("---")
      get_openai_response(model_name, question)

In [ ]:
model_name = "gpt-4o" # teacher
evaluate(model_name,1)

Generating with model: gpt-4o:
---
In touch rugby, a try is typically worth one point. This differs from traditional rugby union, where a try is worth five points. Touch rugby has its own set of rules and scoring system, which often includes simplified scoring to emphasize speed and skill over physical contact.



In [ ]:
model_name = "gpt-4o-mini" # student
evaluate(model_name,1)

Generating with model: gpt-4o-mini:
---
In touch rugby, a try is worth five points.



## Generate and Store Data

In [ ]:
dataset_tag = "touch-rugby-7"

# Function to call OpenAI and print the response
def get_and_store_openai_response(model_name, question, store=True):
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are a sports rules expert."},
            {"role": "user", "content": question}
        ],
        temperature=0.25,
        top_p=0.9,
        metadata={
            "distillation": dataset_tag
        },
        store=store  # Save the completion
    )
    print(f"Response from {model_name} for '{question}':")
    print(response.choices[0].message.content)
    if store:
      print("Response stored on OpenAI servers")
    print()

# Function to generate data and print question numbers
def generate_data(model_name, qs_to_eval=10):
    # Call the function with both models
    for question_number, question in enumerate(train_questions[:qs_to_eval], start=1):
        print(f"Question {question_number}:")
        print("---")
        get_and_store_openai_response(model_name, question)

In [ ]:
model_name = "gpt-4o"
generate_data(model_name)

Question 1:
---
Response from gpt-4o for 'Is a try worth one or five points in touch rugby?':
In touch rugby, a try is typically worth one point. This differs from traditional rugby union, where a try is worth five points. Touch rugby has simplified scoring to emphasize the fast-paced and social nature of the game.
Response stored on OpenAI servers

Question 2:
---
Response from gpt-4o for 'How much is a try worth in touch rugby?':
In touch rugby, a try is worth one point. Unlike traditional rugby union or rugby league, where a try is worth more points, touch rugby simplifies the scoring system with each try counting equally.
Response stored on OpenAI servers

Question 3:
---
Response from gpt-4o for 'Is a try worth five points in touch rugby?':
In touch rugby, a try is typically worth one point. This differs from rugby union, where a try is worth five points. Touch rugby has its own set of rules and scoring system, which emphasizes speed and skill without the physical contact found in

## Fine-tuning and Evaluation
This is best done by going to the fine-tuning view on your OpenAI dashboard.

Then, come back, with your fine-tuned model name in hand, and run evaluation again.

In [ ]:
# model_name = "<your fine-tuned model name>"
model_name = "ft:gpt-4o-mini-2024-07-18:trelis-ltd:touch-rugby-6:AIFAewgZ"
evaluate(model_name,1)

Generating with model: ft:gpt-4o-mini-2024-07-18:trelis-ltd:touch-rugby-6:AIFAewgZ:
---
In touch rugby, a try is typically worth one point. This differs from traditional rugby union, where a try is worth five points. Touch rugby has simplified scoring to emphasize the fast-paced and social nature of the game.

